<a href="https://colab.research.google.com/github/rominarg/equipo22-sentiment-api/blob/main/Proyecto_1_SentimentAPI_%E2%80%94_An%C3%A1lisis_de_Sentimientos_de_Feedbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Twitter**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews-spanish' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews-spanish


In [ ]:
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

# Construct the full path to the CSV file within the downloaded directory
# The previous attempt failed because the file was not found directly in 'path'.
# Let's list the contents of the 'path' directory to identify the correct file name or subdirectory.
print(f"Listing contents of: {path}")
print(os.listdir(path))

# Using the correct CSV file name identified from os.listdir(path)
csv_file_path = os.path.join(path, 'IMDB Dataset SPANISH.csv')

# If the file is within a subfolder, e.g., 'data' inside 'path', adjust like this:
# csv_file_path = os.path.join(path, 'data', 'sentiment_analysis_dataset.csv')

datos = pd.read_csv(csv_file_path)
datos.head()

Listing contents of: /kaggle/input/imdb-dataset-of-50k-movie-reviews-spanish
['IMDB Dataset SPANISH.csv']


,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [ ]:
import pandas as pd
import os

# The 'datos' DataFrame is already loaded from the previous cell (I6wRX4G8MNe5).
# We will now prepare the 'df' DataFrame for subsequent steps from 'datos'.
print("🔄 Preparando DataFrame 'df' a partir de 'datos'...")

# Create a copy of the 'datos' DataFrame to work with
df = datos.copy()

# Rename 'review_es' to 'text' as expected by subsequent cells for text content
df.rename(columns={'review_es': 'text'}, inplace=True)

# Use the 'sentimiento' column as the 'sentiment' column for sentiment labels
# This column contains 'positivo'/'negativo' labels.
df['sentiment'] = df['sentimiento']

# Drop original sentiment columns if no longer needed to avoid confusion
df.drop(columns=['review_en', 'sentimiento', 'Unnamed: 0'], errors='ignore', inplace=True)

print("✅ DataFrame 'df' preparado exitosamente.")

print(f"\n📊 Dimensiones finales: {df.shape[0]} filas x {df.shape[1]} columnas")
print("\n--- Primeras 5 filas ---")
print(df.head())
print("\n--- Nombres de columnas ---")
print(df.columns.tolist())

🔄 Preparando DataFrame 'df' a partir de 'datos'...
✅ DataFrame 'df' preparado exitosamente.

📊 Dimensiones finales: 50000 filas x 2 columnas

--- Primeras 5 filas ---
                                                text sentiment
0  Uno de los otros críticos ha mencionado que de...  positivo
1  Una pequeña pequeña producción.La técnica de f...  positivo
2  Pensé que esta era una manera maravillosa de p...  positivo
3  Básicamente, hay una familia donde un niño peq...  negativo
4  El "amor en el tiempo" de Petter Mattei es una...  positivo

--- Nombres de columnas ---
['text', 'sentiment']


##**Preparación y Etiquetado**

**Lógica de Agrupación:**


**Negativo** (0):  mad 😤 (enojado), sad 😥 (triste), scared 😨 (asustado).


**Positivo**(1):  joyful 😀 (feliz), powerful 💪🏻 (poderoso), peaceful 😌
 (pacífico).

In [ ]:
# Mapeo de sentimientos a binario
mapa_sentimientos = {
    'negativo': 0,      # Negativos
    'positivo': 1      # Positivos
}
df['target'] = df['sentiment'].map(mapa_sentimientos)

df_final = df[['text', 'target']]

In [ ]:
df_final.sample(10)

,text,target
33756,Esta es una de las pocas películas lanzadas so...,1
20980,El reparto estaba bien elegido.Pauly Shore es ...,1
249,'Airport 4' is basically a slopped together me...,0
49327,Imagina a una mujer sola en una casa durante c...,0
17171,Esta película ilustra la peor parte de la guer...,1
36275,Supongo que esta es la primera vez que he vist...,1
30678,Qué premisa fantástica: una película sobre el ...,0
4764,Creo que realmente dejan que la calidad de la ...,0
38994,¡Guau! Acabo de recibir esta película de un am...,0
32388,Le di a esta película un 10/10 con cierta renu...,1


In [ ]:
print("\nConteo de Positivos (1) vs Negativos (0):")
print(df_final['target'].value_counts())


Conteo de Positivos (1) vs Negativos (0):
target
1    25000
0    25000
Name: count, dtype: int64


##**Limpieza de Texto**

**Datos sucios**




In [ ]:
df_final[['text']].sample(10)

,text
30665,"Me gustaría que se le conociera, que a menudo ..."
13544,Los creadores de esta película deben haberse s...
1580,Los franceses hacen películas pro-marxistas o ...
18178,"Pickup en South Street (1953), dirigida por la..."
14865,No puedo ir por mucho tiempo describiendo este...
34809,Cualquiera que piense que esta película no ha ...
30669,Si las buenas intenciones hicieron una películ...
12761,Puntaje final (un promedio de varias cualidade...
475,(Spoilers muy suaves; Un esquema básico de la ...
543,Acabo de ver esta película en DVD a última hor...


In [ ]:
import re

def limpiar_texto(text):
    # 1. Convertir todo a minúsculas (para que "Hola" == "hola")
    text = str(text).lower()

    # 2. Eliminar URLs (http://... o https://...)
    text = re.sub(r'http\S+', '', text)

    # 3. Eliminar menciones (@usuario)
    text = re.sub(r'@\w+', '', text)

    # 4. Eliminar Hashtags (borramos el símbolo # pero dejamos la palabra)
    # Ejemplo: "#NoLaVen" se convierte en "nolaven"
    text = re.sub(r'#', '', text)

    # 5. Eliminar todo lo que NO sea una letra (borra emojis, números y signos)
    text = re.sub(r'[^a-záéíóúñ\s]', '', text)

    # 6. Eliminar espacios dobles (si quedaron huecos vacíos)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Aplicamos la función a toda la columna 'text'
df_final['text_clean'] = df_final['text'].apply(limpiar_texto)
df_final.to_csv("twitter_sentiment_analysis_clean.csv", index=False)

**Datos limpios**

In [ ]:
df_final[['text_clean']].sample(10)

,text_clean
12260,trabajo en una oficina los fines de semana y h...
11445,producido por nott entertainment esta película...
18827,cuando comencé a ver el espectáculo dije oh no...
22436,de hecho este es un espectáculo divertido hech...
6518,una vez más la misma historia familiar sobre u...
37611,me hablé de ver esto por una novia dios era un...
42622,si nunca ha visto esta película y como los ant...
18234,estaba consciente del hombre de las sartenes c...
10440,después del éxito de la segunda cuota richard ...
41029,ok esta es una película extrañalos fanáticos d...


In [ ]:
print("Comparativa: Texto Original vs. Texto Limpio")
print(df_final[['text', 'text_clean']].sample(10, random_state=32))

Comparativa: Texto Original vs. Texto Limpio
                                                    text  \
35083  Corriendo solo setenta y dos minutos, este peq...   
9005   Como tantos experimentos de medios, este esfue...   
23836  Opera de jabón aburrida, absolutamente predeci...   
42777  Esta es una excelente película de Sammo Hung.E...   
13222  En esta película se muestra un mensaje muy pos...   
19069  En la ceremonia de entrega de la Academia el 2...   
28129  Cuando era más joven, y hoy, el Sr. Bean es un...   
10653  This is one of those films where it is easy to...   
37505  Quería ver a Sarah Buffy en la pantalla grande...   
4722   ¡Es showtime! Showtime es simplemente un golpe...   

                                              text_clean  
35083  corriendo solo setenta y dos minutos este pequ...  
9005   como tantos experimentos de medios este esfuer...  
23836  opera de jabón aburrida absolutamente predecib...  
42777  esta es una excelente película de sammo hungen...  

##**Transformación**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


X = df_final["text_clean"]
y= df_final["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline= Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=5000,
        ngram_range=(1,2),
        min_df=5
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])



##**Entrenamiento**

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, min_df=5,
                                 ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

##**Validación**

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = pipeline.predict(X_test)

print(f"--- Resultados del entrenamiento ---")
print(f"Accuracy (Exactitud): {accuracy_score(y_test, y_pred):.2%}")

print("\nReporte detallado:")
print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

--- Resultados del entrenamiento ---
Accuracy (Exactitud): 87.21%

Reporte detallado:
              precision    recall  f1-score   support

    Negativo       0.88      0.87      0.87      5000
    Positivo       0.87      0.88      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



**Prueba**



In [ ]:
mis_frases = [
    "estoy muy feliz con este producto es genial",
    "odio tener que esperar tanto, es horrible",
    "el servicio fue normal, nada especial"
]

mis_predicciones = pipeline.predict(mis_frases)

for frase, prediccion in zip(mis_frases, mis_predicciones):
    resultado = "Positivo" if prediccion == 1 else "Negativo"
    print(f"Texto: '{frase}' -> Predicción: {resultado}")

Texto: 'estoy muy feliz con este producto es genial' -> Predicción: Positivo
Texto: 'odio tener que esperar tanto, es horrible' -> Predicción: Negativo
Texto: 'el servicio fue normal, nada especial' -> Predicción: Negativo


##**Serialización**

In [ ]:
import joblib
joblib.dump(pipeline, 'modelo_sentimiento.joblib')

['modelo_sentimiento.joblib']